In [7]:
%%writefile bayesian_ts_bandit_decay.py
import json
import numpy as np
import pandas as pd

bandit_state = None
total_reward = 0
last_step_bandit = None
opponent_last = None

def bayesian_ts_decay_bandit(observation, configuration):
    global bandit_state,total_reward,last_step_bandit, opponent_last

    if observation.step%100 == 0:
       # print(f"Observations {observation}")
       print(f"Bayesian TS decay bandit {total_reward}")
        
    #if observation.step%1000 == 0:
    #    print(f"Bandit state {bandit_state}")
        
    if observation.step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(100)]
    else:       
        reward = observation.reward - total_reward
        total_reward += reward
        
        # updating bandit_state using the result of the previous step       
        if reward > 0:
            # If winner increase the wins for that particular bandit
            bandit_state[last_step_bandit][0] += reward * 0.97
        
        # Increase the trials - wins for that particular bandit
        bandit_state[last_step_bandit][1] += 1 - reward

        opponent_current = observation.lastActions[1]
        if opponent_last:
            if opponent_last == opponent_current:
                assume_reward = 1 # If opponent repeated perhaps the last time was a win, update alpha
                bandit_state[opponent_last][0] += assume_reward *.97
            else:
                # If opponent did not repeat opponent last was a defeat, update beta
                bandit_state[opponent_last][1] += 1
            
        opponent_last = opponent_current 
    #generate random number from Beta distribution for each agent and select the most lucky one
    #best_agent = np.argmax(np.random.beta(bandit_state[0], bandit_state[1])).item()

    best_proba = -1
    best_agent = None
    for k in range(100):
        proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k

    last_step_bandit = best_agent
    return best_agent

Overwriting bayesian_ts_bandit_decay.py
